In [1]:
role = "system"
content = "You are a helpful assistant."

def construct_chat_message(role: str, content: str) -> dict:
    if role not in ("system", "user", "assistant"):
        raise ValueError("Invalid role, must be 'system', 'user' or 'assistant'.")
    return {"role": role, "content": content}

print(construct_chat_message(role, content))

{'role': 'system', 'content': 'You are a helpful assistant.'}


In [10]:

from typing import Optional, Any
import openai
from time import sleep
import logging

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi there!"}
]

def request_chat_completion(messages: dict, model: str = "gpt-3.5-turbo", max_retries: int = 3, **kwargs: Any) -> Optional[dict]:

    last_error_msg = ""
    for _ in range(max_retries):
        try:
            completion = openai.ChatCompletion.create(
                model=model, messages=messages, **kwargs
            )
            return completion
        except Exception as e:
            last_error_msg = str(e)
            sleep(3)

    logging.error(f"Failed after {max_retries} retries. Last error message: {last_error_msg}")
    return None


print(request_chat_completion(messages=messages, temperature=0.7))

ERROR:root:Failed after 3 retries. Last error message: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000019C9AD85750>: Failed to resolve 'api.openai.com' ([Errno 11001] getaddrinfo failed)"))


None


In [28]:
sys_message = ""
prompt = "Who is Socrates?"

messages = []

if sys_message:
    messages.append(construct_chat_message("system", sys_message))

messages.append(construct_chat_message("user", prompt))

print(messages)

completion = request_chat_completion(messages)

try:
    print(completion.choices[0].message["content"])
except AttributeError as e:
    print(f"Failed to retrieve completion from OpenAI Chat API: {e}")
    raise


[{'role': 'user', 'content': 'Who is Socrates?'}]
Failed after 3 retries. Last error message:
Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001A2F96FB790>: Failed to resolve 'api.openai.com' ([Errno 11001] getaddrinfo failed)"))
Failed to retrieve completion from OpenAI Chat API. 'NoneType' object has no attribute 'choices'


In [16]:
import logging
from typing import Any
import openai

sys_message = "You are a helpful assistant answering like a pirate"
prompt = "Who is Socrates?"

def get_chat_response(prompt: str, sys_message: str = "", model: str = "gpt-3.5-turbo", **kwargs: Any) -> str:
    """
    Generates a chatbot response using OpenAI's Chat API.
    
    Args:
        prompt (str): The message from the user.
        sys_message (str, optional): A system message for the LLM. Defaults to an empty string.
        model (str, optional): The name of the OpenAI model to use. Defaults to "gpt-3.5-turbo".
        **kwargs: Additional keyword arguments to pass to the `request_chat_completion` function.
        
    Returns:
        Optional[str]: The content of the LLM's response or None if an error occurs.

    Raises:
        TypeError: Raised when `request_chat_completion` returns `None`.
    """
    messages = []
    if sys_message:
        messages.append(construct_chat_message("system", sys_message))
        
    messages.append(construct_chat_message("user", prompt))
    completion = request_chat_completion(messages, model=model, **kwargs)

    try:
        logging.info("Token usage: %s", completion['usage'])
        return completion.choices[0].message["content"]
    except TypeError as e:
        logging.error("Failed to retrieve completion from OpenAI Chat API: %s", {e})
        raise

print(get_chat_response(prompt, sys_message, temperature = 0.7))

Arr, Socrates be an ancient Greek philosopher, matey! He be known fer his deep thinkin' and his fancy way o' questionin' everything. He be one o' the founders o' Western philosophy, teachin' many famous lads like Plato and Xenophon. Socrates be a wise ol' soul who be sailin' the seas o' knowledge!
